In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#!pip install import-ipynb
%cd /content/drive/My\ Drive/Colab\ Notebooks/RetinaSmartCamera/notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/notebooks


In [ ]:
#!pip install pyzmq
!pip install dippykit

In [ ]:
import time
import zmq

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:8888")

while True:
    #  Wait for next request from client
    message = socket.recv()
    print(f"Received request: {message}")

    #  Do some 'work'
    time.sleep(1)

    #  Send reply back to client
    socket.send(b"World")

KeyboardInterrupt: ignored

In [ ]:
import import_ipynb
import RetinaUtils as ru

import numpy as np
from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode

import cv2
import re
import os
from sys import getsizeof

importing Jupyter notebook from RetinaUtils.ipynb


In [ ]:
import cv2
def imshow(img):
    x, y = img.shape[0], img.shape[1]
    plt.figure()
    plt.imshow(img, cmap='gray' if len(img.shape)==2 else None)
    plt.axis('off')

img_path = '/content/drive/My Drive/Personal Dataset/dock.jpg'
img_bgr = cv2.imread(img_path)
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_gray = cv2.imread(img_path, 0)

fixation = (1920/2, 1080/2)
data_dir = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/retina_data/'
Retina = ru.Retina(gpu=False)
Retina.loadLoc(data_dir+'ret8k_loc.pkl')
Retina.loadCoeff(data_dir+'ret8k_coeff.pkl')
Retina.prepare(img.shape, fixation)

Cortex = ru.Cortex(gpu=False)
cortex_dir = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/retina_data/cortex_data/'

Cortex.loadLocs(cortex_dir+'8k_cort_leftloc.pkl', cortex_dir+'8k_cort_rightloc.pkl')
Cortex.loadCoeffs(cortex_dir+'8k_cort_leftcoeff.pkl', cortex_dir+'8k_cort_rightcoeff.pkl')

In [ ]:
V = Retina.sample(img, fixation)
original_cort = Cortex.cort_img(V)
imshow(original_cort)
#imshow(original_cort.astype('uint32'))

flattened = V.reshape(-1)
print(V.dtype, V.shape)
seq, bits, code_dict, _ = huffman_encode(flattened.astype('uint32'))

In [ ]:
decoded = huffman_decode(seq, code_dict)
print(decoded.shape)
unflattened = decoded.reshape((8192, 3))
imshow(Cortex.cort_img(unflattened))